In [1]:
import numpy as np
import Utils as ut

In [2]:
from sklearn import datasets
# make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, 
#                     n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
#                     hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
ds, label = datasets.make_classification(n_samples=300, n_features=30, n_informative=10, n_redundant=0, n_classes=2)

In [3]:
np.where(label==0)
label[np.where(label==0)]=-1

In [4]:
def random_sampling(X, y, n_samples):
    """Extract n_samples from X and y."""
    from sklearn.model_selection import StratifiedShuffleSplit
    
    return next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))
     

In [5]:
tr_idx, ts_idx = random_sampling(ds, label, n_samples=0.25)


ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = ds[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

l_tr = label[tr_idx]
l_ts = label[ts_idx]

In [6]:
kernel_types = ['linear', 'polynomial', 'gaussian']
k_dataset_wrapper = []
k_train_list = []
k_test_list = []

for train, test in zip([ds1_tr, ds2_tr, ds3_tr], [ds1_ts, ds2_ts, ds3_ts]):
    for k_type in kernel_types:
        if k_type=='polynomial': 
            current = ut.kernel(train, K_type=k_type, param=3)
        else:
            current = ut.kernel(train, K_type=k_type, param=0.5)
            
        k_dataset_wrapper.append({'kernel':current, 'train_ds':train, 'test_ds':test})
        k_train_list.append(current.kernelMatrix(train))
        k_test_list.append(current.kernelMatrix(test))


In [ ]:
weights = ut.centeredKernelAlignment(k_train_list, l_tr)
print(weights)

In [7]:
#idealK_overTr = np.dot(l_tr.reshape(-1,1), l_tr.reshape(-1,1).T)
new_kernels_config = ut.parameterOptimization(k_dataset_wrapper, l_tr, tol=0.001, verbose=True)

epoch num 1; current CA is: -0.142518744145224
	optimizing 5
		optimizing over [0.3,0.6000000000000001]
		selected 0.3 with sim: 0.4915986964294047
	optimizing 0
	optimizing 4
		optimizing over [1,5]
		selected 1 with sim: 0.4977816929487604
	optimizing 7
		optimizing over [1,5]
		kept 3 with sim: 0.4977816929487604
	optimizing 1
		optimizing over [1,5]
		kept 3 with sim: 0.4977816929487604
	optimizing 6
	optimizing 3
	optimizing 8
		optimizing over [0.3,0.6000000000000001]
		kept 0.5 with sim: 0.4977816929487604
	optimizing 2
		optimizing over [0.3,0.6000000000000001]
		kept 0.5 with sim: 0.4977816929487604
epoch num 2; current CA is: 0.033886736144827144
	optimizing 2
		optimizing over [0.3,0.6000000000000001]
		selected 0.3 with sim: 0.08494842149508869
	optimizing 4
		optimizing over [1,3]
		kept 1 with sim: 0.08494842149508869
	optimizing 6
	optimizing 0
	optimizing 1
		optimizing over [1,5]
		selected 4 with sim: 0.499641664902832
	optimizing 7
		optimizing over [1,5]
		kept 3 wi

In [ ]:
pred = np.zeros(len(ts_idx))
for k_test, w in zip(k_test_list, weights):
    weighted_labeled_kernel = np.multiply(k_test, l_tr*w)  
    pred += np.sum(weighted_labeled_kernel, axis=1)
pred = np.sign(pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(l_ts, pred)

In [ ]:
a = np.array([[1,2],[3,4]])
print(a.ravel().reshape(-1,1).shape)